# 데이터프레임에서 활용가능한 최적화 방법
## 파일형식에 따른 데이터프레임 로딩 및 저장 테스트
큰 사이즈의 데이터를 다루면 파일 형식에 따른 데이터프레임 로딩 속도 및 원본데이터의 용량 차이가 존재 하게 됩니다.  
실무에서 많이 사용하는 xlsx, csv 형식과 함께 parquet 형태의 데이터를 알아봅니다.

parquet 데이터 파일은 행단위 데이터 누적이 아닌 열(컬럼)단위 데이터 압축을 하는 특징으로 빅데이터 로딩 및 관리에 효율적인 방법입니다. 
<img src="./image/parquet.png">  
출처 : https://www.dremio.com/resources/guides/intro-apache-parquet/

In [1]:
# 필요모듈 import
import numpy as np
from numpy import uint8
import pandas as pd
pd.options.display.max_columns = 100

In [34]:
%%time
# 엑셀파일 로딩 테스트 'ERDTSUM_DATA_AGG.xlsx'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99548 entries, 0 to 99547
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   eqp_model_rev    99548 non-null  object
 1   eqp_id_rev       99548 non-null  object
 2   area             99548 non-null  object
 3   eqp_type         99548 non-null  object
 4   ees_type         274 non-null    object
 5   unit_name_rev    99548 non-null  object
 6   disp_name        99548 non-null  object
 7   lottype          99548 non-null  object
 8   wafer_id         99548 non-null  int64 
 9   param_name_rev   99548 non-null  object
 10  act_time         99548 non-null  object
 11  lower_spec_rev   99548 non-null  object
 12  param_value_rev  99548 non-null  object
 13  upper_spec_rev   99548 non-null  object
 14  step_seq_rev     99548 non-null  object
 15  processid_rev    99548 non-null  object
 16  partid_rev       99548 non-null  object
 17  ppid_rev         99548 non-null

In [39]:
%%time
# xlsx


CPU times: user 38.7 s, sys: 1.02 s, total: 39.7 s
Wall time: 40.1 s


In [40]:
%%time
# csv 테스트용 csv파일 생성


CPU times: user 10.4 s, sys: 452 ms, total: 10.8 s
Wall time: 11 s


In [41]:
%%time
# parquet


CPU times: user 1.07 s, sys: 247 ms, total: 1.31 s
Wall time: 1.34 s


In [35]:
%%time
# csv 파일 로딩 테스트


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99548 entries, 0 to 99547
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   eqp_model_rev    99548 non-null  object
 1   eqp_id_rev       99548 non-null  object
 2   area             99548 non-null  object
 3   eqp_type         99548 non-null  object
 4   ees_type         99548 non-null  object
 5   unit_name_rev    99548 non-null  object
 6   disp_name        99548 non-null  object
 7   lottype          99548 non-null  object
 8   wafer_id         99548 non-null  int64 
 9   param_name_rev   99548 non-null  object
 10  act_time         99548 non-null  object
 11  lower_spec_rev   99548 non-null  object
 12  param_value_rev  99548 non-null  object
 13  upper_spec_rev   99548 non-null  object
 14  step_seq_rev     99548 non-null  object
 15  processid_rev    99548 non-null  object
 16  partid_rev       99548 non-null  object
 17  ppid_rev         99548 non-null

In [89]:
%%time
# parquet 파일 로딩 테스트


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99548 entries, 0 to 99547
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   eqp_model_rev    99548 non-null  object
 1   eqp_id_rev       99548 non-null  object
 2   area             99548 non-null  object
 3   eqp_type         99548 non-null  object
 4   ees_type         99548 non-null  object
 5   unit_name_rev    99548 non-null  object
 6   disp_name        99548 non-null  object
 7   lottype          99548 non-null  object
 8   wafer_id         99548 non-null  int64 
 9   param_name_rev   99548 non-null  object
 10  act_time         99548 non-null  object
 11  lower_spec_rev   99548 non-null  object
 12  param_value_rev  99548 non-null  object
 13  upper_spec_rev   99548 non-null  object
 14  step_seq_rev     99548 non-null  object
 15  processid_rev    99548 non-null  object
 16  partid_rev       99548 non-null  object
 17  ppid_rev         99548 non-null

동일한 데이터프레임으로 로딩하지만 작업시간, 데이터의 크기가 파일형식마다 특징이 있습니다.  
parquet데이터의 경우 속도 및 파일용량 면에서 타 파일형식보다 월등한 효율을 갖습니다.  

# 데이터타입 최적화
판다스의 데이터타입은 기본적으로 numpy array데이터 타입을 따르며 array에 담겨있는 데이터 중 가장 큰 범주의 데이터를 기준으로 데이터타입을 사용합니다.  
eg.) str > float < int  
데이터타입은 컴퓨터 운영체제의  비트수를 따르며(일반적으로 64bit) 일부 윈도우 컴퓨터는 메모리 최적화를 위해 데이터의 범위에 따라 데이터타입을 정의하기도 합니다.  
다만 데이터의 사이즈가 클 경우 전체 데이터를 로컬 컴퓨터의 메모리상에 다 담을 수 없어 작업을 위해 추가적인 조치가 필요합니다.  
각 데이터의 고윳값 특징에 따라 데이터타입을 정의한다면 컬럼별 약 2~8배의 메모리 절약이 가능합니다.  
큰 정밀도가 필요하지 않은 경우 int64, float64 --> 작은 메모리 int32, float16, uint8 등등  
object --> category

## 데이터 타입별 비트수, 표현 가능한 숫자범위
|데이터타입|비트수|숫자범위|
|---|---|---|
|int16|16비트|-32767~32768|
|int32|32비트|-2147483648 ~ 2147483647|
|int64|64비트|-9223372036854775808 ~ 9223372036854775807|
|unint8|8비트|0~255|
|uint16|16비트|0~65535|
|uint32|32비트|0 ~ 4294967295|
|uint64|64비트|0 ~ 18446744073709551615|

|데이터타입|비트수|숫자범위|소수점 이하 정밀도|
|---|---|---|---|
|float16(반정밀도)|16비트|$-6.55\times10^4$ ~ $6.55\times10^4$|약 3~4자리|
|float32(단정밀도)|32비트|$-3.4\times10^38$ ~ $3.4\times10^38$|약 6~7자리|
|float64(배정밀도)|64비트|$-1.8\times10^308$ ~ $1.8x10\times308$|약 15~16자리|

In [43]:
# 데이터형식 확인


,eqp_model_rev,eqp_id_rev,area,eqp_type,ees_type,unit_name_rev,disp_name,lottype,wafer_id,param_name_rev,act_time,lower_spec_rev,param_value_rev,upper_spec_rev,step_seq_rev,processid_rev,partid_rev,ppid_rev,recipeid_rev,root_lot_id_rev,lotid_rev,fdc_bin_rev,fdc_model_rev
0,HCD9300,A79C01,METRO,METRMS,METRMS,A79C01,0,-,0,"[""d_awrpcelloutvoltage_e"",""d_bandwidth_e"",""d_c...","[""2024-01-30 00:03:59"",""2024-01-30 00:03:59"",""...","[11.57002159338028,,,,0,13.20025437969012,,,,0...","[0,0,0,0,0,0]","[24.40612206853535,,,,8.71363057312702,24.7986...",SVMMYSQEP,-,SAMPLE,PBP_12BLO_PPE,-,131915,131915PP,0,OV_YBPM300_MP_PPOXLE
1,HCD9300,A79C01,METRO,METRMS,METRMS,A79C01,0,-,0,"[""v_rrc_rinse_flow_e""]","[""2024-01-30 00:07:45"",""2024-01-30 00:07:45""]",[],"[-2.282096333333333,-3.279004,-1.2817383,0.673...",[],SVMMYSQEP,-,SAMPLE,PBP_12BLO_PPE,-,119670,119670P,0,OV_YBPM300_MP_PPOXLE
2,HCD9300,A79C01,METRO,METRMS,METRMS,A79C01,0,-,0,"[""v_smas_si2h7_mfc_si2h7_avg_avg_t"",""v_smas_97...","[""2024-01-30 00:17:59"",""2024-01-30 00:17:59"",""...","[182.0792824601367,,,]","[1.27,0.2899999999999977,0.2899999999999977,0,...","[184.0792824601367,,,]",SVMMYSQEP,-,SAMPLE,PBP_12BLO_PPE,-,131915,131915PP,0,OV_YBPM300_MP_PPOXLE
3,HCD9300,A79C01,METRO,METRMS,METRMS,A79C01,0,-,0,"[""plate_temp_max_min_t"",""v_process_time_avg_t""...","[""2024-01-30 00:26:31"",""2024-01-30 00:26:31"",""...","[,,0,,,0]","[15.244,15.244,15.244,0,16.222,16.222,16.222,0...","[,,100,,,100]",SVMMYSQEP,-,SAMPLE,PBP_12BLO_PPE,-,103981,103981P,0,OV_YBPM300_MP_PPOXLE
4,89C_PX_OX7DQ_D73GPQ,C8OC07,CMP,CMPOXD,-,C8OC07-DR,DR,PP,16,"[""v_step_out_detect_time_avg_t"",""v_step_out_de...","[""2024-01-30 00:00:00""]","[,,,]","[1.239393939393939,0,5,5,2.281219565049078,172...","[,,,]",EM055240,LBEM,L9XBXS8V0F-BXQ,EMP1V,-,BMOB80,BMOB80.1,SI,EMQ_LL_OXPSE_PPOXLE


## 데이터타입 최적화를 위한 스키마 확인 및 설정
### 카테고리 데이터 컬럼명 추출

In [74]:
# 전체데이터를 메모리에 로딩이 가능한 경우 변수명에 따라 변수의 데이터타입 확인 (카테고리 여부 확인)
# 고윳값 갯수가 20개 미만일 경우 카테고리컬화
# 카테고리 컬럼을 선별 후 딕셔너리에 {컬럼명:'category'}형식으로 저장 


area 10 object
['METRO' 'CMP' 'DIFF' 'IMP' 'ETCH' 'METAL' 'PHOTO' 'CVD' 'AMHS' 'CLN']
ees_type 10 object
['METRMS' '-' 'Sorter_KT' 'METRMC' 'METRCD' 'METRTK' 'METRDI' 'METROD'
 'POVLAY' 'METRPC']
lottype 3 object
['-' 'PP' 'EE']
processid_rev 12 object
['-' 'LBEM' 'LEOE' 'LBEP' 'LBEL' 'LEOS' 'LEOV' 'LEOB' 'LEVE' 'LEQL' 'LEQF'
 'IM0ES0']
partid_rev 13 object
['SAMPLE' 'L9XBXS8V0F-BXQ' '-' 'L4E4X085YE-EEF' 'L9XBXS8V0E-BXQ'
 'L9XSXS8V0S-BXQ' 'L4B4X0846E-EEF' 'L4V2E164EF-ELX' 'L4F2E164LF-EEB'
 'L4E8X085YB-EEF' 'L9X00B8M0S-EXQ' 'L4B4X1646S-EEF' 'L4B2X1646F-EEF']


In [80]:
# 전체 데이터 로딩이 불가능한 경우 컬럼데이터 하나씩만 호출하여 고윳값 확인 (데이터타입 지정을 위한 고윳값 확인)
# csv, xlsx만 가능 parquet 불가
# usecols=[col_nm] 으로 시리즈형태의 데이터로 로드
# wafer_id의 경우 정수형태 데이터타입이 int --> 데이터의 범위가 255 미만


 39%|██████████████████▊                             | 9/23 [00:19<00:30,  2.18s/it]

wafer_id [ 0 16 22 14  5 21 24 23 37 15 20 13 49 10 41 12  8  7  1  3 52  6  2  9
  4 33 19 51 18 17 11 54 28 39 59 40 31 35 58 50 27 30 38 53 43 46 34 29
 55 42 48 45 56 25 47 44 36 26 32 57 99] int64 796516


100%|███████████████████████████████████████████████| 23/23 [00:55<00:00,  2.39s/it]


In [81]:
# uint8변환 데이터 딕셔너리에 추가


In [82]:
# 데이터 확인


{'area': 'category',
 'ees_type': 'category',
 'lottype': 'category',
 'processid_rev': 'category',
 'partid_rev': 'category',
 'wafer_id': numpy.uint8}

In [91]:
# 원본데이터에 데이터 타입 적용 후 메모리 사용량 확인 memory_usage


원본데이터 컬럼별 메모리 사용량
Index                    132
eqp_model_rev        6458454
eqp_id_rev           5474757
area                 5257677
eqp_type             5475527
ees_type             4978887
unit_name_rev        5925915
disp_name            5254604
lottype              5056585
wafer_id              796384
param_name_rev     495447937
act_time           419663938
lower_spec_rev      50466405
param_value_rev    179800454
upper_spec_rev      54826143
step_seq_rev         5554082
processid_rev        5214957
partid_rev           6016715
ppid_rev             5958946
recipeid_rev         5600855
root_lot_id_rev      5473151
lotid_rev            5715785
fdc_bin_rev          5224604
fdc_model_rev        7152794
dtype: int64

변환 후 컬럼별 메모리 사용량

Index                    132
eqp_model_rev        6458454
eqp_id_rev           5474757
area                  100377
eqp_type             5475527
ees_type              100396
unit_name_rev        5925915
disp_name            5254604
lottype               

In [94]:
# parquet 저장 시 데이터타입 설정 engine='pyarrow'
# parquet 로딩 시 dtype_backend='pyarrow'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99548 entries, 0 to 99547
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype                                                       
---  ------           --------------  -----                                                       
 0   eqp_model_rev    99548 non-null  string[pyarrow]                                             
 1   eqp_id_rev       99548 non-null  string[pyarrow]                                             
 2   area             99548 non-null  dictionary<values=string, indices=int32, ordered=0>[pyarrow]
 3   eqp_type         99548 non-null  string[pyarrow]                                             
 4   ees_type         99548 non-null  dictionary<values=string, indices=int32, ordered=0>[pyarrow]
 5   unit_name_rev    99548 non-null  string[pyarrow]                                             
 6   disp_name        99548 non-null  string[pyarrow]                                             


## 파일 로딩 작업 모듈화 클래스 작성 실습
위 데이터타입 최적화 과정을 함수 형태로 제작하고 클래스화 시켜보도록 하겠습니다.

In [12]:
# 이전 작업 클래스 가져오기
# 메소드 실행 시 
# 고윳값 갯수가 20개 이하인 컬럼 선별 후 카테고리 데이터 타입으로 변환
# 정수형 데이터 선별, 데이터 범위 255까지 확인 후 uint8 타입으로 변환
# 데이터프레임 반환


In [13]:
# 모듈 파일 실행 후 info()로 데이터타입 확인


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99548 entries, 0 to 99547
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   eqp_model_rev    99548 non-null  object  
 1   eqp_id_rev       99548 non-null  object  
 2   area             99548 non-null  category
 3   eqp_type         99548 non-null  object  
 4   ees_type         99548 non-null  category
 5   unit_name_rev    99548 non-null  object  
 6   disp_name        99548 non-null  object  
 7   lottype          99548 non-null  category
 8   wafer_id         99548 non-null  uint8   
 9   param_name_rev   99548 non-null  object  
 10  act_time         99548 non-null  object  
 11  lower_spec_rev   99548 non-null  object  
 12  param_value_rev  99548 non-null  object  
 13  upper_spec_rev   99548 non-null  object  
 14  step_seq_rev     99548 non-null  object  
 15  processid_rev    99548 non-null  category
 16  partid_rev       99548 non-null  categor